In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --upgrade transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 77.6 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 

## Packages

In [ ]:
hf_token = ""
from huggingface_hub import login

login(hf_token)

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig, Trainer, TrainingArguments
import torch
from datasets import load_dataset
import pandas as pd
from peft import get_peft_model, prepare_model_for_kbit_training, LoraConfig, TaskType, LoraModel
from datasets import Dataset
from torch.utils.data import DataLoader

2025-05-02 14:22:36.087278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746195756.326580      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746195756.391046      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
ds1 = load_dataset("JasleenSingh91/travel-questions-response")
df1 = pd.DataFrame(ds1['train'])
input1 = df1["Input"]
output1 = df1["Response_2"]
print(f"input: {input1.shape}, output: {output1.shape}")

input-response1-response2.csv:   0%|          | 0.00/20.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6009 [00:00<?, ? examples/s]

input: (6009,), output: (6009,)


In [5]:
ds2 = load_dataset("Binaryy/travel_sample_extended")
df2 = pd.DataFrame(ds2['train'])
input2 = df2["query"]
output2 = df2["response"]
print(f"input: {input2.shape}, output: {output2.shape}")

README.md:   0%|          | 0.00/398 [00:00<?, ?B/s]

(…)-00000-of-00001-d5eea51e61dddc73.parquet:   0%|          | 0.00/110k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/110 [00:00<?, ? examples/s]

input: (110,), output: (110,)


In [6]:
res1 = '''
Switzerland boasts a plethora of breathtaking destinations, catering to diverse interests. Here are some of the best places to visit:

**Cities & Towns:**

* **Zurich:** Switzerland's largest city, a global hub with a blend of modern and historic charm. Explore the Old Town (Altstadt) with its winding lanes, visit numerous museums and art galleries, enjoy a boat trip on Lake Zurich, and experience the lively nightlife.
* **Bern:** The capital city, a UNESCO World Heritage site, known for its well-preserved medieval Old Town with charming arcades, historic fountains, and the Zytglogge clock tower. Don't miss the Einstein House and the Rose Garden for panoramic views.
* **Lucerne:** A picturesque city on Lake Lucerne, surrounded by mountains. Admire the iconic Chapel Bridge, the Lion Monument, and enjoy the stunning views from Mount Pilatus or Mount Rigi. It's a perfect base for exploring Central Switzerland.
* **Interlaken:** Situated between Lake Thun and Lake Brienz, it's known as the "Adventure Capital" offering activities like paragliding, canyoning, and hiking. It provides stunning views of the Jungfrau massif.
* **Geneva:** A cosmopolitan city on Lake Geneva, home to international organizations like the United Nations. Visit the Jet d'Eau fountain, explore the Old Town, and enjoy the lakeside promenades. It's also a gateway to the Lavaux vineyards.
* **Zermatt:** A car-free village nestled at the foot of the Matterhorn. It's a world-renowned destination for skiing and mountaineering, offering breathtaking views of the iconic peak. Take the Gornergrat Railway for unparalleled vistas.
* **Lausanne:** An elegant university city on the shores of Lake Geneva, offering beautiful lake views and a rich cultural heritage. Visit the Lausanne Cathedral and the Olympic Museum.
* **Lugano:** Located in the Italian-speaking part of Switzerland, it offers a Mediterranean vibe with a mix of Swiss and Italian culture. Enjoy boat rides on Lake Lugano and explore the charming Parco Ciani.
* **Montreux:** A charming town on Lake Geneva, famous for its mild climate and the Freddie Mercury statue. Visit the stunning Chillon Castle and enjoy the views of the terraced vineyards of Lavaux.
* **Grindelwald:** A picturesque alpine village in the Bernese Alps, offering stunning scenery and outdoor adventures. It's a great base for exploring the Jungfrau region.
* **Lauterbrunnen:** A stunning valley with 72 waterfalls, surrounded by towering cliffs. The Staubbach Falls are particularly impressive. It's a gateway to the Jungfrau region and car-free villages like Wengen and Mürren.
* **Stein am Rhein:** A beautifully preserved medieval town on the Rhine River, known for its half-timbered buildings with colorful painted facades.

**Natural Wonders & Attractions:**

* **The Matterhorn:** An iconic pyramid-shaped mountain in the Alps, a must-see landmark.
* **Jungfraujoch - Top of Europe:** Home to the highest railway station in Europe, offering breathtaking views of glaciers and the surrounding peaks.
* **Lake Geneva (Lac Léman):** Switzerland's largest lake, offering stunning views, lakeside towns, and vineyards.
* **Lake Lucerne (Vierwaldstättersee):** A beautiful lake surrounded by mountains, offering scenic boat trips and access to various mountain excursions.
* **Rhine Falls (Rheinfall):** One of Europe's largest waterfalls, a powerful and impressive natural spectacle.
* **Aletsch Glacier:** The largest glacier in the Alps, a UNESCO World Heritage site.
* **Swiss National Park:** The oldest and largest nature reserve in Switzerland, offering opportunities for wildlife observation and hiking.
* **Oeschinensee:** A stunning mountain lake near Kandersteg, accessible by gondola.
* **Creux du Van:** A natural rock arena in the Jura Mountains, offering panoramic views.
* **Staubbach Falls:** One of the highest free-falling waterfalls in Europe, located in the Lauterbrunnen Valley.
* **Trümmelbach Falls:** A series of ten glacier-fed waterfalls inside a mountain in the Lauterbrunnen Valley.

**Scenic Train Rides:**

* **Glacier Express:** A panoramic train journey connecting Zermatt and St. Moritz, traversing stunning alpine landscapes.
* **Bernina Express:** A spectacular train ride through the Alps, connecting Chur and Tirano, passing over impressive viaducts and through tunnels.
* **GoldenPass Line:** A scenic route connecting Lucerne and Montreux, offering diverse landscapes from lakes to mountains.
* **Gotthard Panorama Express:** A journey combining a boat trip on Lake Lucerne and a train ride over the historic Gotthard railway line.

When planning your trip, consider your interests (hiking, skiing, culture, relaxation), the time of year, and your budget to tailor your itinerary to the best places for you.

'''

res2 = '''
Certainly I can for a better suggestion. Can you give me some personal details such as are do you have place in mind? Are you going alone or with friends, family, partner.
Do you plan to stay in a hotel?
What kind of budget do have in mind?
Are you planning to travel with a car or public transport.
What weather conditions do you prefer?
For example, this is winter time so, so if you are planning a cozy winter plan. You can go to Switzerland it has some amazing places to visit.
It has largest lake, offering stunning views, lakeside towns, and vineyards.
You can Skii on the icy mountains and enjoy stunning mountanious view from the top of the mountain.
You can ride cable cars.
Please let me know so i can plan my trip more specifically.
'''

res3 = '''
Considering your preferences for a warm tropical country with excellent summer weather, beautiful beaches with clean water, stunning green mountains, luxurious ocean-view accommodations, abundant activities, and great food, I highly recommend a trip to Maui, Hawaii, USA.

Here's a potential 7-day travel plan for you and your wife, keeping in mind that "summer" in the Northern Hemisphere (where Hawaii is located) typically runs from June to August, offering warm and sunny weather:

Destination: Maui, Hawaii, USA

Duration: 7 Days / 6 Nights

Best Time to Visit (for your preference): June to August (warmest and driest weather)

Accommodation: The Ritz-Carlton, Kapalua (renowned for its luxurious ocean-view rooms, stunning location, and exceptional service) or the Four Seasons Resort Maui at Wailea (another top-tier option with breathtaking views and amenities).

Day 1: Arrival in Maui & Settling into Paradise

Fly into Kahului Airport (OGG) on Maui.
Upon arrival, you'll be greeted with the fragrant scent of tropical flowers – a classic Hawaiian welcome.
Private transfer to your chosen luxurious hotel in Kapalua or Wailea. Both locations offer stunning ocean views and upscale amenities.
Check in and take some time to settle into your opulent ocean-view suite. Enjoy the view and the hotel's facilities.
Evening: Enjoy a romantic welcome dinner at Merriman's Kapalua or Spago Maui, both known for their exquisite Hawaiian-inspired cuisine with oceanfront settings.
Day 2: Exploring West Maui's Beauty

Morning: After a leisurely breakfast, embark on a scenic drive along the West Maui coastline.
Visit Kaanapali Beach, famous for its golden sands and clear turquoise waters. Relax on the beach, go for a swim, or try stand-up paddleboarding.
Afternoon: Explore Lahaina Town, a historic whaling village with art galleries, boutiques, and waterfront restaurants.
Consider a whale watching tour (if visiting between November and May, though the water clarity is excellent year-round).
Evening: Witness a breathtaking sunset from the cliffs of Kapalua Point, followed by dinner at Star Noodle, a local favorite with Asian-inspired dishes.
Day 3: The Majestic Road to Hana

Full Day: Embark on the iconic Road to Hana, a scenic drive with lush rainforests, cascading waterfalls, dramatic coastal cliffs, and stunning ocean vistas.
Stop at various points of interest along the way, such as:
Twin Falls: An easily accessible waterfall for a refreshing dip.
Garden of Eden Arboretum: A beautiful botanical garden showcasing Hawaiian flora.
Waianapanapa State Park: Known for its black sand beach, sea caves, and blowholes.
Hamoa Beach: A picturesque crescent-shaped beach.
Pools of Ohe'o (Seven Sacred Pools): A series of cascading waterfalls and pools (check for accessibility as it can sometimes be closed due to weather).
Pack a picnic lunch to enjoy amidst the natural beauty or dine at a local eatery in Hana.
Evening: Return to your hotel, perhaps indulging in a relaxing couples massage.
Day 4: Underwater Adventures & Relaxation

Morning: Take a snorkeling or scuba diving trip to the Molokini Crater, a partially submerged volcanic crater teeming with marine life and crystal-clear waters. Numerous tour operators offer excursions from Ma'alaea Harbor.
Afternoon: Return to your hotel and enjoy the luxurious amenities. Relax by the pool, unwind on the beach, or indulge in spa treatments.
Evening: Enjoy a fine dining experience at Mama's Fish House, a legendary restaurant known for its fresh, locally sourced seafood and Polynesian ambiance (reservations highly recommended well in advance).
Day 5: Exploring Upcountry Maui & Volcanic Wonders

Morning: Drive upcountry to explore the cooler, greener side of Maui.
Visit the Maui Tropical Plantation, a historic agricultural estate with shops, restaurants, and ziplining opportunities.
Continue to Kula, known for its flower farms, botanical gardens, and panoramic views of the island.
Afternoon: Drive to the summit of Haleakala National Park, a dormant volcano offering breathtaking views of the lunar-like landscape. Consider arriving in time for the sunset (reservations often required).
Evening: Enjoy dinner at a restaurant in Kula with stunning views or head back down to the coast for more dining options.
Day 6: Water Sports & Island Delights

Morning: Indulge in some water sports activities. Consider:
Surfing lessons: Maui has some excellent beginner-friendly surf spots.
Kayaking or outrigger canoeing: Explore the coastline at your own pace.
Sailing charter: Enjoy a leisurely sail along the coast.
Afternoon: Explore the charming town of Paia, known for its eclectic shops, art galleries, and laid-back vibe. Enjoy lunch at one of its many cafes.
Evening: Experience a traditional Hawaiian luau with Polynesian food, music, and dance for a cultural immersion. The Old Lahaina Luau is highly recommended.
Day 7: Farewell to Paradise

Morning: Enjoy a final leisurely breakfast with stunning ocean views.
Depending on your flight schedule, you might have time for some last-minute souvenir shopping or a final dip in the ocean or pool.
Private transfer to Kahului Airport (OGG) for your departure, filled with unforgettable memories of your luxurious and adventurous tropical getaway.
Food Options:

Maui offers a diverse culinary scene:

Fresh Seafood: From upscale restaurants like Mama's Fish House to casual beachfront grills.
Hawaiian Regional Cuisine: A fusion of local ingredients and international influences.
Farm-to-Table: Many restaurants emphasize fresh, locally sourced produce.
Food Trucks: Offering a variety of delicious and affordable options.
Local Specialties: Don't miss trying poi, laulau, kalua pig, and shave ice.
Activities:

Beyond the itinerary, consider these activities:

Ziplining: Experience the thrill of soaring through the rainforest canopy.
Hiking: Maui offers trails for all levels, from coastal walks to challenging mountain treks.
Golfing: World-class golf courses with stunning ocean views.
Horseback Riding: Explore the island's interior on horseback.
Helicopter Tours: Get a bird's-eye view of Maui's dramatic landscapes.
Art Galleries and Museums: Explore the local culture and art scene.
This itinerary is a suggestion and can be customized to your interests and pace. Maui offers a perfect blend of natural beauty, luxurious accommodations, exciting activities, and delicious food, making it an ideal destination for a memorable tropical getaway for you and your wife. Enjoy your trip!
'''

## Preparing Data

In [7]:
df3 = pd.DataFrame({
    "Input": ["What are the best places to visit in Switzerland?", "Can you plan a day trip for me?",
              "Make a travel plan for me and my wife on a warm tropical country with good summer weather. I want a place with nice beaches and clean water also beautiful green mountain. I have no budget problem but I want a nice view hotel with luxurious room with nice ocean view. Also there are lots of activity to do in my trip and great food options"],
    "Response": [res1, res2, res3]
})

In [8]:
all_input = pd.concat([input1, input2, df3["Input"]])
all_input = all_input.reset_index(drop=True)
all_output = pd.concat([output1, output2, df3["Response"]])
all_output = all_output.reset_index(drop=True)

# all_input = pd.concat([input2, df3["Input"]])
# all_input = all_input.reset_index(drop=True)
# all_output = pd.concat([output2, df3["Response"]])
# all_output = all_output.reset_index(drop=True)

## Creating Model

In [28]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
q_config = BitsAndBytesConfig(load_in_4bit=True) 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config,
    # quantization_config = q_config,
    torch_dtype = torch.float16,
    # device_map="auto"
)
model = prepare_model_for_kbit_training(model)
# model.to("cuda")

## Creating Prompt

In [10]:
def prompt(example):
    prompt = f"""
    <|user|> {example["input"]}

    <|bot|> {example["output"]}
    
    """
     
    p_input = tokenizer(prompt, max_length=1024, truncation=True, padding="max_length")
    p_input["labels"] = p_input["input_ids"].copy()
    return p_input

ds = Dataset.from_pandas(pd.DataFrame({
    "input": all_input,
    "output": all_output
}))

split_dataset = ds.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

train_dataset = train_dataset.map(prompt)
eval_dataset = train_dataset.map(prompt)

Map:   0%|          | 0/5509 [00:00<?, ? examples/s]

Map:   0%|          | 0/5509 [00:00<?, ? examples/s]

## Lora Config

In [29]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.02,
    target_modules=["q_proj", "k_proj", "o_proj"],
    use_rslora = True
)

lora_model = get_peft_model(model ,lora_config)

## Training Config

In [30]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/results/lora",
    eval_strategy = "steps",
    num_train_epochs = 3,
    per_device_train_batch_size = 8,  # micro batching
    gradient_accumulation_steps = 8,
    logging_dir="./logs",  # Logging directory
    save_steps = 100,  # How often to save the model
    fp16=True,  # Use mixed precision training
    # device = torch.device("cuda")
    report_to="none",
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

/tmp/ipykernel_31/2224886751.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [22]:
lora_model


In [31]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
lora_model

In [ ]:
lora_model.save_pretrained("/kaggle/working/fine_tuned_llama_qlora")
tokenizer.save_pretrained("/kaggle/working/fine_tuned_llama_qlora")